In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('labeledTrainData.tsv', header = 0, delimiter = '\t', quoting=3)
df.shape

(25000, 3)

In [3]:
df.head()

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."
3,"""3630_4""",0,"""It must be assumed that those who praised thi..."
4,"""9495_8""",1,"""Superbly trashy and wondrously unpretentious ..."


In [4]:
df.describe()

,sentiment
count,25000.00000
mean,0.50000
std,0.50001
min,0.00000
25%,0.00000
50%,0.50000
75%,1.00000
max,1.00000


In [5]:
df.review[0]

'"With all this stuff going down at the moment with MJ i\'ve started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ\'s feeling towards the press and also the obvious message of drugs are bad m\'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally

In [6]:
from bs4 import BeautifulSoup as bs
import re
import nltk
from nltk.corpus import stopwords

In [7]:
def transformReview(rawReview):
    """
    This function takes a raw string review as input, and applies the following steps to return a refined string review as output:
    1. removing HTML tags
    2. removing punctuation marks
    3. converting the text to lower case
    4. splitting the string into words
    5. removing stop words
    """
    # remove HTML tags
    noHtml = bs(rawReview, "lxml").get_text()
    
    #remove punctuation marks
    letters_only = re.sub("[^a-zA-Z]", " ", noHtml)
    
    #convert to lower case
    tolower = letters_only.lower()
    
    #split
    words = tolower.split()
    
    #convert stopwords list to set for fast searching
    stopwordsSet = set(stopwords.words("english"))
    
    #remove stop words
    RefinedWords = [w for w in words if w not in stopwordsSet]
    
    #form new review
    return(" ".join(RefinedWords) )

In [8]:
clean_review = transformReview(df.review[0])
clean_review

'stuff going moment mj started listening music watching odd documentary watched wiz watched moonwalker maybe want get certain insight guy thought really cool eighties maybe make mind whether guilty innocent moonwalker part biography part feature film remember going see cinema originally released subtle messages mj feeling towards press also obvious message drugs bad kay visually impressive course michael jackson unless remotely like mj anyway going hate find boring may call mj egotist consenting making movie mj fans would say made fans true really nice actual feature film bit finally starts minutes excluding smooth criminal sequence joe pesci convincing psychopathic powerful drug lord wants mj dead bad beyond mj overheard plans nah joe pesci character ranted wanted people know supplying drugs etc dunno maybe hates mj music lots cool things like mj turning car robot whole speed demon sequence also director must patience saint came filming kiddy bad sequence usually directors hate workin

In [9]:
sze = len(df.review)
clean_review = []

for i in range(sze):
    clean_review.append( transformReview(df.review[i]))

In [10]:
# form vocabulary and select top 5000 words
print("creating bag of words...\n")

from sklearn.feature_extraction.text import CountVectorizer

vectorized = CountVectorizer(analyzer='word', max_features=5000)

creating bag of words...



In [11]:
print("extracting features from bag of words...")
train_features = vectorized.fit_transform(clean_review)
train_features = train_features.toarray()
train_features.shape

extracting features from bag of words...


(25000, 5000)

In [12]:
vocab = vectorized.get_feature_names()
# vocab

In [22]:
# See the frequency of each feature in train_features
dist = np.sum(train_features, axis = 0)
print(dist.shape)

# print vocab word and its frequency
# for word, frq in zip(vocab, dist):
#     print(word + "\t" + str(frq))

(5000,)


In [14]:
# training random forest
print("training random forest...\n")
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators = 100)
rf = rf.fit(train_features, df.sentiment)

training random forest...



/home/shreya/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [15]:
# make predicitons on test dataset

# load
test_data = pd.read_csv("testData.tsv", delimiter="\t", header= 0, quoting=3)
print(test_data.shape)
print(test_data.columns)

# transform all reviews in the test data
n_reviews = test_data.shape[0]
clean_reviews = []

print('Cleaning reviews...\n')
for i in range(0, n_reviews):
    clean_reviews.append(transformReview(test_data.review[i]))

test_data_vector = vectorized.transform(clean_reviews)
test_data_vector = test_data_vector.toarray()

# predict using random forest
test_pred = rf.predict(test_data_vector)


(25000, 2)
Index(['id', 'review'], dtype='object')
Cleaning reviews...



In [21]:
# output dataframe
output = pd.DataFrame( data = {"id" : test_data["id"], 'sentiment': test_pred})

# convert to csv
output.to_csv("BoW_model.csv", index = False, quoting = 3)

In [17]:
output.shape
output.head()

,id,review
0,"""12311_10""",1
1,"""8348_2""",0
2,"""5828_4""",1
3,"""7186_2""",1
4,"""12128_7""",1


In [18]:
test_data.head()

,id,review
0,"""12311_10""","""Naturally in a film who's main themes are of ..."
1,"""8348_2""","""This movie is a disaster within a disaster fi..."
2,"""5828_4""","""All in all, this is a movie for kids. We saw ..."
3,"""7186_2""","""Afraid of the Dark left me with the impressio..."
4,"""12128_7""","""A very accurate depiction of small time mob l..."


In [19]:
df.head()

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."
3,"""3630_4""",0,"""It must be assumed that those who praised thi..."
4,"""9495_8""",1,"""Superbly trashy and wondrously unpretentious ..."


In [20]:
sample = pd.read_csv("sampleSubmission.csv")
sample.head()
# type(sample.id[0])

,id,sentiment
0,12311_10,0
1,8348_2,0
2,5828_4,0
3,7186_2,0
4,12128_7,0
